In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import DataFrame
import datetime
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold # 追加

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/restaurant-revenue-prediction/leaderboard.csv
/kaggle/input/restaurant-revenue-prediction/train.csv
/kaggle/input/restaurant-revenue-prediction/sampleSubmission.csv
/kaggle/input/restaurant-revenue-prediction/test.csv


In [2]:
df_trainval = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/train.csv')
df_test = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/test.csv')
print(df_trainval.head())

   Id   Open Date        City  City Group Type  P1   P2   P3   P4  P5  ...  \
0   0  07/17/1999    İstanbul  Big Cities   IL   4  5.0  4.0  4.0   2  ...   
1   1  02/14/2008      Ankara  Big Cities   FC   4  5.0  4.0  4.0   1  ...   
2   2  03/09/2013  Diyarbakır       Other   IL   2  4.0  2.0  5.0   2  ...   
3   3  02/02/2012       Tokat       Other   IL   6  4.5  6.0  6.0   4  ...   
4   4  05/09/2009   Gaziantep       Other   IL   3  4.0  3.0  4.0   2  ...   

   P29  P30  P31  P32  P33  P34  P35  P36  P37    revenue  
0  3.0    5    3    4    5    5    4    3    4  5653753.0  
1  3.0    0    0    0    0    0    0    0    0  6923131.0  
2  3.0    0    0    0    0    0    0    0    0  2055379.0  
3  7.5   25   12   10    6   18   12   12    6  2675511.0  
4  3.0    5    1    3    2    3    4    3    3  4316715.0  

[5 rows x 43 columns]


In [3]:
rev = df_trainval['revenue']
del df_trainval['revenue']

In [4]:
df_all = pd.concat([df_trainval,df_test],axis=0)
df_all['Open Date'] = pd.to_datetime(df_all["Open Date"])
df_all['Year'] = df_all['Open Date'].apply(lambda x:x.year)
df_all['Month'] = df_all['Open Date'].apply(lambda x:x.month)
df_all['Day'] = df_all['Open Date'].apply(lambda x:x.day)

le = LabelEncoder()
df_all['City'] = le.fit_transform(df_all['City'])
df_all['City Group'] = df_all['City Group'].map({'Other':0,'Big Cities':1})
df_all["Type"] = df_all["Type"].map({"FC":0, "IL":1, "DT":2, "MB":3})
print(df_all.head())

   Id  Open Date  City  City Group  Type  P1   P2   P3   P4  P5  ...  P31  \
0   0 1999-07-17    60           1     1   4  5.0  4.0  4.0   2  ...    3   
1   1 2008-02-14     4           1     0   4  5.0  4.0  4.0   1  ...    0   
2   2 2013-03-09    14           0     1   2  4.0  2.0  5.0   2  ...    0   
3   3 2012-02-02    52           0     1   6  4.5  6.0  6.0   4  ...   12   
4   4 2009-05-09    21           0     1   3  4.0  3.0  4.0   2  ...    1   

   P32  P33  P34  P35  P36  P37  Year  Month  Day  
0    4    5    5    4    3    4  1999      7   17  
1    0    0    0    0    0    0  2008      2   14  
2    0    0    0    0    0    0  2013      3    9  
3   10    6   18   12   12    6  2012      2    2  
4    3    2    3    4    3    3  2009      5    9  

[5 rows x 45 columns]


In [5]:
df_trainval = df_all.iloc[:df_trainval.shape[0]]
df_test = df_all.iloc[df_trainval.shape[0]:]

In [6]:
df_train_col = [col for col in df_trainval.columns if col not in ['Id','Open Date']]

In [7]:
sc = StandardScaler()
ms = MinMaxScaler()

In [8]:
df_trainval_sc = sc.fit_transform(df_trainval[df_train_col])
df_trainval_sc_ms = ms.fit_transform(df_trainval_sc)
print(DataFrame(df_trainval_sc_ms).head())

         0    1    2         3         4         5         6         7   \
0  0.967742  1.0  0.5  0.272727  0.615385  0.533333  0.222222  0.142857   
1  0.064516  1.0  0.0  0.272727  0.615385  0.533333  0.222222  0.000000   
2  0.225806  0.0  0.5  0.090909  0.461538  0.266667  0.444444  0.142857   
3  0.838710  0.0  0.5  0.454545  0.538462  0.800000  0.666667  0.428571   
4  0.338710  0.0  0.5  0.181818  0.461538  0.400000  0.222222  0.142857   

         8         9   ...        33    34        35        36        37  \
0  0.111111  0.444444  ...  0.200000  0.16  0.833333  0.208333  0.266667   
1  0.111111  0.444444  ...  0.000000  0.00  0.000000  0.000000  0.000000   
2  0.222222  0.444444  ...  0.000000  0.00  0.000000  0.000000  0.000000   
3  0.333333  1.000000  ...  0.800000  0.40  1.000000  0.750000  0.800000   
4  0.111111  0.444444  ...  0.066667  0.12  0.333333  0.125000  0.266667   

     38     39        40        41        42  
0  0.15  0.500  0.166667  0.545455  0.533333 

In [9]:
def gen_cv():
    m_train = np.floor(len(rev)*0.75).astype(int)#このキャストをintにしないと後にハマる
    train_indices = np.arange(m_train)
    test_indices = np.arange(m_train, len(rev))
    yield (train_indices, test_indices)

In [10]:
params_cnt = 20
params = {"C":np.logspace(0,1,params_cnt), "epsilon":np.logspace(-1,1,params_cnt)}
gridsearch = GridSearchCV(SVR(kernel="linear"), params, cv=gen_cv(), scoring="r2", return_train_score=True)
gridsearch.fit(df_trainval_sc_ms, rev)
print('The best parameter = ',gridsearch.best_params_)
print('accuracy = ',gridsearch.best_score_)
print()

regr = SVR(kernel="linear", C=gridsearch.best_params_["C"], epsilon=gridsearch.best_params_["epsilon"])

The best parameter =  {'C': 10.0, 'epsilon': 0.1}
accuracy =  -0.029186054898893588



In [11]:
splits = 5
kf = KFold(n_splits=splits,shuffle=True,random_state=0)

mlp = MLPRegressor(activation='relu',
                  solver='adam',
                  batch_size=100,
                  max_iter=2000,
                   hidden_layer_sizes=(16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,)
                  )
rf = RandomForestRegressor(n_estimators=1000,
                           max_depth=30,
                           random_state=0,
                           n_jobs=-1)

rmse_list = []
#models_mlp = []
#models_rf = []
#models_svr = []
for train_index,test_index in kf.split(df_trainval_sc_ms):
    X_train = df_trainval.iloc[train_index]
    Y_train = rev.iloc[train_index]
    X_valid = df_trainval.iloc[test_index]
    Y_valid = rev.iloc[test_index]
    model_mlp = mlp.fit(X_train[df_train_col], Y_train)
    model_rf = rf.fit(X_train[df_train_col], Y_train)
    model_svr = regr.fit(X_train[df_train_col], Y_train)
    models_mlp.append(model_mlp)
    models_rf.append(model_rf)
    models_svr.append(model_svr)
    prediction_rf = rf.predict(X_valid[df_train_col])
    prediction_mlp = mlp.predict(X_valid[df_train_col])
    prediction_regr = regr.predict(X_valid[df_train_col])
    prediction = (prediction_rf + prediction_mlp + prediction_regr) / 3
    val_rmse = mean_absolute_error(Y_valid,prediction)
    print(val_rmse)
    rmse_list.append(val_rmse)
print('average rmse : {0}'.format(sum(rmse_list)/len(rmse_list)))

NameError: name 'models_mlp' is not defined

In [12]:
df_test_sc = sc.transform(df_test[df_train_col])
df_test_sc_ms = ms.fit_transform(df_test_sc)

In [13]:
mlp.fit(df_trainval_sc_ms, rev)
rf.fit(df_trainval_sc_ms, rev)
regr.fit(df_trainval_sc_ms, rev)

prediction_rf = rf.predict(df_test_sc_ms)
prediction_mlp = mlp.predict(df_test_sc_ms)
prediction_regr = regr.predict(df_test_sc_ms)

prediction = (prediction_rf + prediction_mlp + prediction_regr) / 3

submission = DataFrame({'Id':df_test.Id,'Prediction':prediction})

In [14]:
submission.to_csv('./submission191019.csv',index=False)
!ls && pwd

__notebook__.ipynb  __output__.json  submission191019.csv
/kaggle/working
